# Synthesizer

Synthesizer ใช่เพื่อสำหรับการ generate data เพื่อ evaluation โดยใช้ LLM ในการ generate input 

In [ ]:
import sys
sys.path.append('/opt/project/src/evaluate_llm/')
from api_key_config import settings

In [1]:
from langchain_openai import AzureChatOpenAI
from deepeval.models.base_model import DeepEvalBaseLLM

class AzureOpenAI(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return "Custom Azure OpenAI Model"

# Replace these with real values
custom_model = AzureChatOpenAI(
    openai_api_version=settings.OPENAI_API_VERSION,
    azure_deployment="gpt-35-turbo",
    azure_endpoint=settings.AZURE_OPENAI_ENDPOINT,
    openai_api_key=settings.OPENAI_API_KEY,
)
azure_openai = AzureOpenAI(model=custom_model)

/usr/local/lib/python3.9/site-packages/deepeval/__init__.py:42: UserWarning: You are using deepeval version 0.21.42, however version 0.21.45 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


การใช้ synthesizer จะแบ่งออกเป็น 2 รูปแบบ
1. สร้างจาก context ที่ถูก extract จาก document 
2. สร้าง list ที่ provide context ให้ 

### Generating From Documents

พารามิเตอร์ generate_goldens_from_docs :
- document_paths
- include_expected_output ต้องการให้สร้างคำตอบไหม default = Fasle 
- max_goldens_per_document จำนวนจุดข้อมูล golden point ที่จะสร้างสำหรับแต่ละเอกสาร
- chunk_size
- chunk_overlap
- num_evolutions ควบคุมความซับซ้อนและความหลากหลายของชุดข้อมูลที่สร้างขึ้นโดยการปรับปรุงและพัฒนาอินพุตเริ่มต้นซ้ำๆ default 1
- enable_breadth_evolve เมื่อตั้งค่าเป็น True จะทำให้เกิดการปรับเปลี่ยนบริบทที่หลากหลายมากขึ้น ซึ่งช่วยเพิ่มความหลากหลายของชุดข้อมูล

In [ ]:
from deepeval.synthesizer import Synthesizer

synthesizer = Synthesizer()
synthesizer.generate_goldens_from_docs(
    document_paths=['example.txt', 'example.docx', 'example.pdf'],
    max_goldens_per_document=2
)

### Generating From Provided Contexts

พารามิเตอร์ของ generate_goldens :
- contexts: รายการของบริบท ซึ่งแต่ละบริบทเป็นรายการของสตริงที่ควรมีธีมหรือพื้นที่เนื้อหาที่เหมือนกัน
- [ตัวเลือก] include_expected_output: เมื่อกำหนดเป็น True จะสร้าง expected_output สำหรับแต่ละ synthetic Golden เพิ่มเติม ค่าเริ่มต้นคือ False
- [ตัวเลือก] max_goldens_per_context: จำนวนสูงสุดของจุดข้อมูล golden ที่จะสร้างจากแต่ละบริบท การปรับพารามิเตอร์นี้สามารถมีผลต่อขนาดของชุดข้อมูลที่ได้ ค่าเริ่มต้นคือ 2
- [ตัวเลือก] num_evolutions: จำนวนขั้นตอน evolution ที่จะใช้กับอินพุตที่สร้างแต่ละอัน พารามิเตอร์นี้ควบคุมความซับซ้อนและความหลากหลายของชุดข้อมูลที่สร้างโดยการปรับแต่งและวิวัฒนาการอินพุตเริ่มต้นซ้ำ ๆ ค่าเริ่มต้นคือ 1
- [ตัวเลือก] enable_breadth_evolve: ระบุว่าจะเปิดใช้งาน breadth evolution strategies ระหว่างการสร้างข้อมูลหรือไม่ เมื่อกำหนดเป็น True จะมีการนำการแก้ไขบริบทที่หลากหลายมากขึ้นมาใช้ เพิ่มความหลากหลายของชุดข้อมูล ค่าเริ่มต้นคือ False


In [4]:
from deepeval.synthesizer import Synthesizer

synthesizer = Synthesizer(model = azure_openai)
synthesizer.generate_goldens(
    # Provide a list of context for synthetic data generation
    contexts=[
        ["The Earth revolves around the Sun.", "Planets are celestial bodies."],
        ["Water freezes at 0 degrees Celsius.", "The chemical formula for water is H2O."],
    ],
    include_expected_output = True,
    max_goldens_per_context = 2
)

Output()

[Golden(input='What celestial body does the Earth revolve around in our solar system?', actual_output=None, expected_output='The Earth revolves around the Sun in our solar system.', context=['The Earth revolves around the Sun.', 'Planets are celestial bodies.'], retrieval_context=None, additional_metadata=None, comments=None, source_file=None),
 Golden(input="What are the implications of the Earth's orbit around the Sun on the celestial bodies in the solar system?", actual_output=None, expected_output="The Earth's orbit around the Sun affects the motion of other celestial bodies in the solar system.", context=['The Earth revolves around the Sun.', 'Planets are celestial bodies.'], retrieval_context=None, additional_metadata=None, comments=None, source_file=None),
 Golden(input='How does the freezing point of water compare to its chemical formula?', actual_output=None, expected_output='The freezing point of water is 0 degrees Celsius, and its chemical formula is H2O.', context=['Water f

In [ ]:
saved_path = dataset.save_as(
    file_type='json',  # or 'csv'
    directory="./synthetic_data"
)